In [ ]:
"""
Simple Face Attendance System
==============================
A straightforward face recognition attendance system using MTCNN and FaceNet.
"""

import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import os
import pickle
from pathlib import Path

from mtcnn import MTCNN
from keras_facenet import FaceNet
from scipy.spatial.distance import cosine


# =============================================================================
# CONFIGURATION
# =============================================================================
DATA_DIR = Path("attendance_data")
EMBEDDINGS_FILE = DATA_DIR / "users.pkl"
ATTENDANCE_FILE = DATA_DIR / "attendance.csv"

RECOGNITION_THRESHOLD = 0.6  # Lower = stricter
MIN_FACE_SIZE = 80

# Create data directory
DATA_DIR.mkdir(exist_ok=True)


# =============================================================================
# INITIALIZE MODELS
# =============================================================================
print("Loading models...")
face_detector = MTCNN()
face_recognizer = FaceNet()
print("✓ Models loaded!")


# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def detect_face(image):
    """Detect and return the largest face in the image"""
    detections = face_detector.detect_faces(image)
    
    if not detections:
        return None
    
    # Get largest face
    detection = max(detections, key=lambda x: x['box'][2] * x['box'][3])
    x, y, w, h = detection['box']
    
    # Check minimum size
    if w < MIN_FACE_SIZE or h < MIN_FACE_SIZE:
        return None
    
    # Extract face with padding
    padding = int(0.2 * max(w, h))
    x1 = max(0, x - padding)
    y1 = max(0, y - padding)
    x2 = min(image.shape[1], x + w + padding)
    y2 = min(image.shape[0], y + h + padding)
    
    face = image[y1:y2, x1:x2]
    
    # Resize to 160x160 (required by FaceNet)
    face = cv2.resize(face, (160, 160))
    
    return face, (x, y, w, h)


def get_embedding(face_image):
    """Get face embedding vector"""
    # Convert to RGB
    face_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    face_rgb = np.expand_dims(face_rgb, axis=0)
    
    # Get embedding
    embedding = face_recognizer.embeddings(face_rgb)[0]
    
    # Normalize
    embedding = embedding / np.linalg.norm(embedding)
    
    return embedding


def load_users():
    """Load registered users from file"""
    if EMBEDDINGS_FILE.exists():
        with open(EMBEDDINGS_FILE, 'rb') as f:
            return pickle.load(f)
    return {}


def save_users(users):
    """Save registered users to file"""
    with open(EMBEDDINGS_FILE, 'wb') as f:
        pickle.dump(users, f)


def init_attendance_file():
    """Initialize attendance CSV file"""
    if not ATTENDANCE_FILE.exists():
        df = pd.DataFrame(columns=['User', 'Date', 'Punch-in', 'Punch-out'])
        df.to_csv(ATTENDANCE_FILE, index=False)


# =============================================================================
# MAIN FUNCTIONS
# =============================================================================

def register_user(username):
    """Register a new user using camera"""
    print(f"\n📸 Registering: {username}")
    print("Position your face in the camera and press 'c' to capture")
    print("Press 'q' to cancel\n")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Cannot open camera")
        return False
    
    registered = False
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        display = frame.copy()
        result = detect_face(frame)
        
        if result:
            face, (x, y, w, h) = result
            cv2.rectangle(display, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(display, "Press 'c' to capture", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(display, "No face detected", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow('Registration', display)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('c') and result:
            # Get embedding
            embedding = get_embedding(face)
            
            # Save user
            users = load_users()
            users[username] = embedding
            save_users(users)
            
            print(f"✓ {username} registered successfully!")
            registered = True
            break
            
        elif key == ord('q'):
            print("Registration cancelled")
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    return registered


def recognize_face(face_image):
    """Recognize a face and return username"""
    users = load_users()
    
    if not users:
        return None
    
    # Get embedding for the face
    embedding = get_embedding(face_image)
    
    # Find best match
    best_match = None
    best_similarity = 0
    
    for username, stored_embedding in users.items():
        similarity = 1 - cosine(embedding, stored_embedding)
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = username
    
    # Check threshold
    if best_similarity >= (1 - RECOGNITION_THRESHOLD):
        return best_match, best_similarity
    
    return None


def mark_attendance(username, confidence):
    """Mark attendance for a user"""
    init_attendance_file()
    
    current_date = datetime.now().strftime("%Y-%m-%d")
    current_time = datetime.now().strftime("%H:%M:%S")
    
    # Load attendance
    df = pd.read_csv(ATTENDANCE_FILE)
    
    # Check today's record
    today = df[(df['User'] == username) & (df['Date'] == current_date)]
    
    if today.empty:
        # First time today - Punch in
        new_row = pd.DataFrame([{
            'User': username,
            'Date': current_date,
            'Punch-in': current_time,
            'Punch-out': ''
        }])
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(ATTENDANCE_FILE, index=False)
        
        message = f"✓ PUNCH-IN: {username} at {current_time} ({confidence:.1%})"
        
    else:
        # Check if already punched out
        last = today.iloc[-1]
        
        if pd.isna(last['Punch-out']) or last['Punch-out'] == '':
            # Punch out
            idx = df[(df['User'] == username) & 
                    (df['Date'] == current_date) &
                    (df['Punch-out'].isna() | (df['Punch-out'] == ''))].index[-1]
            
            df.at[idx, 'Punch-out'] = current_time
            df.to_csv(ATTENDANCE_FILE, index=False)
            
            message = f"✓ PUNCH-OUT: {username} at {current_time}"
        else:
            # New punch-in
            new_row = pd.DataFrame([{
                'User': username,
                'Date': current_date,
                'Punch-in': current_time,
                'Punch-out': ''
            }])
            df = pd.concat([df, new_row], ignore_index=True)
            df.to_csv(ATTENDANCE_FILE, index=False)
            
            message = f"✓ PUNCH-IN (again): {username} at {current_time}"
    
    return message


def run_attendance():
    """Run the attendance system"""
    print("\n🎥 Starting attendance system...")
    print("Look at the camera to mark attendance")
    print("Press 'q' to quit\n")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Cannot open camera")
        return
    
    last_recognition = {}
    cooldown = 3  # seconds
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        display = frame.copy()
        result = detect_face(frame)
        
        if result:
            face, (x, y, w, h) = result
            recognition = recognize_face(face)
            
            if recognition:
                username, confidence = recognition
                
                # Check cooldown
                import time
                current_time = time.time()
                
                if username not in last_recognition or \
                   (current_time - last_recognition[username]) > cooldown:
                    
                    # Mark attendance
                    message = mark_attendance(username, confidence)
                    print(message)
                    last_recognition[username] = current_time
                    
                    color = (0, 255, 0)
                    label = f"{username} ({confidence:.1%})"
                else:
                    color = (0, 255, 255)
                    label = f"{username} (wait...)"
            else:
                color = (0, 0, 255)
                label = "Unknown"
            
            cv2.rectangle(display, (x, y), (x+w, y+h), color, 2)
            cv2.putText(display, label, (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        cv2.putText(display, "Press 'q' to quit", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Face Attendance', display)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


def view_attendance():
    """View today's attendance"""
    init_attendance_file()
    
    current_date = datetime.now().strftime("%Y-%m-%d")
    df = pd.read_csv(ATTENDANCE_FILE)
    today = df[df['Date'] == current_date]
    
    print("\n" + "="*60)
    print(f"ATTENDANCE for {current_date}")
    print("="*60)
    
    if today.empty:
        print("No records for today")
    else:
        print(today.to_string(index=False))
    
    print("="*60 + "\n")


def list_users():
    """List all registered users"""
    users = load_users()
    
    print("\n" + "="*60)
    print(f"REGISTERED USERS ({len(users)})")
    print("="*60)
    
    if users:
        for i, username in enumerate(users.keys(), 1):
            print(f"{i}. {username}")
    else:
        print("No users registered")
    
    print("="*60 + "\n")


# =============================================================================
# MAIN MENU
# =============================================================================

def main():
    """Main program"""
    print("="*60)
    print("FACE ATTENDANCE SYSTEM")
    print("="*60)
    
    while True:
        print("\nMENU:")
        print("1. Register new user")
        print("2. Start attendance")
        print("3. View today's attendance")
        print("4. List registered users")
        print("5. Exit")
        
        choice = input("\nChoice (1-5): ").strip()
        
        if choice == '1':
            username = input("Enter username: ").strip()
            if username:
                register_user(username)
        
        elif choice == '2':
            run_attendance()
        
        elif choice == '3':
            view_attendance()
        
        elif choice == '4':
            list_users()
        
        elif choice == '5':
            print("Goodbye!")
            break
        
        else:
            print("Invalid choice!")


if __name__ == "__main__":
    main()

Loading models...
✓ Models loaded!
FACE ATTENDANCE SYSTEM

MENU:
1. Register new user
2. Start attendance
3. View today's attendance
4. List registered users
5. Exit

📸 Registering: Meet
Position your face in the camera and press 'c' to capture
Press 'q' to cancel

1/1 [==============================] - 1s 931ms/step
✓ Meet registered successfully!

MENU:
1. Register new user
2. Start attendance
3. View today's attendance
4. List registered users
5. Exit

🎥 Starting attendance system...
Look at the camera to mark attendance
Press 'q' to quit

1/1 [==============================] - 0s 23ms/step
✓ PUNCH-IN: Meet at 22:40:49 (54.2%)
1/1 [==============================] - 0s 23ms/step
✓ PUNCH-OUT: Meet at 22:40:52
1/1 [==============================] - 0s 14ms/step


C:\Users\Harsh\AppData\Local\Temp\ipykernel_16132\2148574574.py:239: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '22:40:52' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'Punch-out'] = current_time


1/1 [==============================] - 0s 22ms/step
✓ PUNCH-IN (again): Meet at 22:41:00
1/1 [==============================] - 0s 24ms/step

MENU:
1. Register new user
2. Start attendance
3. View today's attendance
4. List registered users
5. Exit

📸 Registering: kai
Position your face in the camera and press 'c' to capture
Press 'q' to cancel

1/1 [==============================] - 0s 23ms/step
✓ kai registered successfully!

MENU:
1. Register new user
2. Start attendance
3. View today's attendance
4. List registered users
5. Exit

🎥 Starting attendance system...
Look at the camera to mark attendance
Press 'q' to quit

1/1 [==============================] - 0s 22ms/step
✓ PUNCH-IN: kai at 22:42:47 (44.1%)
1/1 [==============================] - 0s 23ms/step
✓ PUNCH-OUT: kai at 22:42:50
1/1 [==============================] - 0s 24ms/step
✓ PUNCH-IN (again): kai at 22:42:57
1/1 [==============================] - 0s 14ms/step

MENU:
1. Register new user
2. Start attendance
3. View today'